In [1]:
import os
from pathlib import Path
from PIL import Image, ImageDraw
import re
import numpy as np

## First solution

In [2]:
# Check if filenames are equal in both the image and annotations folder
image_names = os.listdir('../data/images')
annotations = os.listdir('../data/kitti_annotations')

file_image_no_extension = [re.sub(r'\.jpg$', '', image_name) for image_name in image_names]
file_annotations_no_extension = [re.sub(r'\.txt$', '', filename) for filename in annotations]

print(set(file_image_no_extension)==set(file_annotations_no_extension))

set()
set()


In [3]:
# Function to scale the bounding box
def scale_bounding_box(width, height, bounding_box, target_size):
    # Compute scaled width and height
    width_scale = target_size/width
    height_scale = target_size/height

    # Compute scaled bounding box coordinates
    x_min_scale = int(np.round(bounding_box[0] * width_scale))
    y_min_scale = int(np.round(bounding_box[1] * height_scale))
    x_max_scale = int(np.round(bounding_box[2] * width_scale))
    y_max_scale = int(np.round(bounding_box[3] * height_scale))
    
    return x_min_scale, y_min_scale, x_max_scale, y_max_scale

In [4]:
# Static variables
filenames = file_image_no_extension
target_size = 284
target_folder = '../output'

# Create output folder
if not Path(os.fspath(target_folder)).exists():
    Path(os.fspath(target_folder)).mkdir()

Path(os.fspath(os.path.join(target_folder, 'images'))).mkdir()
Path(os.fspath(os.path.join(target_folder, 'annotations'))).mkdir()
        
# Iterate over all filenames
for filename in filenames:
    
    # Paths to image and annotations folder
    path_to_annotations = os.path.join('../data/kitti_annotations', filename +'.txt')
    path_to_image = os.path.join('../data/images', filename+'.jpg')
    
    # Paths to image and annotations scaled folder
    path_to_scaled_annotations = os.path.join(target_folder, 'annotations', filename+'.txt')
    path_to_scaled_image = os.path.join(target_folder, 'images', filename+'.jpg')
    
    # Create image and get width and height
    image = Image.open(path_to_image)
    width, height = image.size
    
    # Open annotation file and get all object annotations
    with open(path_to_annotations, 'r') as file:
        object_annotations = file.readlines()
    
     
    with open(path_to_scaled_annotations, 'w') as file:
        
        for obj in object_annotations:
            
            labels = re.findall(r'[-]*[0-9]+[.]*[0-9]*', obj)

            bounding_box_coord = [int(label) for label in labels[3:7]]
            x_min_scale, y_min_scale, x_max_scale, y_max_scale = scale_bounding_box(width, height, bounding_box_coord, target_size)
            labels[3:7] = x_min_scale, y_min_scale, x_max_scale, y_max_scale
            object_scaled = ' '.join(str(label) for label in labels)
            annotation_scaled = re.findall(r'[a-zA-Z]+', obj)[0]+' '+object_scaled
            
            file.write(annotation_scaled+'\n')
        
    
    # Scale and save image
    image.resize((target_size, target_size))   
    image.save(path_to_scaled_image)
    